In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


def split_data_with_window(x_in, y_in, split_window_size):
    # Initialize lists to store training and temporary sets
    x_out1_list, y_out1_list, x_out2_list, y_out2_list = [], [], [], []

    # Iterate through the data with the specified window size
    for i in range(0, len(x_in) - split_window_size, split_window_size + 1):
        x_out1_out2 = x_in.iloc[i:i+split_window_size+1]
        y_out1_out2 = y_in.iloc[i:i+split_window_size+1]

        # Separate the last row for the temporary set
        # [ :-1]: all elements except the last one
        # [-1:]:  selects only the last element
        # (:) is used to indicate slicing of a sequence
        # sequence[start : end : step]

        x_out1 = x_out1_out2.iloc[:-1]
        y_out1 = y_out1_out2.iloc[:-1]

        x_out2 = x_out1_out2.iloc[-1:]
        y_out2 = y_out1_out2.iloc[-1:]

        x_out1_list.append(x_out1)
        y_out1_list.append(y_out1)
        x_out2_list.append(x_out2)
        y_out2_list.append(y_out2)

    # Concatenate the lists into pandas DataFrames
    x_out1 = pd.concat(x_out1_list)
    y_out1 = pd.concat(y_out1_list)
    x_out2 = pd.concat(x_out2_list)
    y_out2 = pd.concat(y_out2_list)

    return x_out1, y_out1, x_out2, y_out2










# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)



# Split Data to train and temp

# Define your split_window_size
split_window_size = 3

# Call the split_data_with_window function
x_train, y_train, x_temp, y_temp = split_data_with_window(x_data, y_data, split_window_size)


"""

# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))

"""

# Split temp into val and test

# Define your split_window_size
split_window_size = 1

# Call the split_data_with_window function
x_val, y_val, x_test, y_test = split_data_with_window(x_temp, y_temp, split_window_size)




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""









class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [24]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [4]
num_layers_list = [2]
learning_rates = [0.0002]
window_sizes = [5]

num_epochs = 1200
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=4, num_layers=2, learning_rate=0.0002, window_size=5


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1200], Training Loss: 28388.47489208361, Validation Loss: 28115.943359375
Epoch [2/1200], Training Loss: 27150.39558028046, Validation Loss: 26965.1640625
Epoch [3/1200], Training Loss: 26077.423699468287, Validation Loss: 25931.212890625
Epoch [4/1200], Training Loss: 25121.48970722734, Validation Loss: 25010.828125
Epoch [5/1200], Training Loss: 24278.404660282653, Validation Loss: 24199.33203125
Epoch [6/1200], Training Loss: 23541.802013532822, Validation Loss: 23489.986328125
Epoch [7/1200], Training Loss: 22903.069461783372, Validation Loss: 22874.099609375
Epoch [8/1200], Training Loss: 22351.941646400464, Validation Loss: 22341.681640625
Epoch [9/1200], Training Loss: 21877.441564977253, Validation Loss: 21882.197265625
Epoch [10/1200], Training Loss: 21468.771943953478, Validation Loss: 21485.3984375
Epoch [11/1200], Training Loss: 21116.033277628892, Validation Loss: 21142.080078125
Epoch [12/1200], Training Loss: 20810.623113635855, Validation Loss: 20844.056640625


In [75]:
import numpy as np
import torch

def predict_future(model, initial_input, num_future_steps, scaler, device='cpu'):
    model.eval()
    future_predictions = []

    # Convert initial input to tensor and move to device
    # Ensure initial input has the shape (1, window_size, input_size)
    input_seq = torch.tensor(initial_input, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        for _ in range(num_future_steps):
            output = model(input_seq)
            future_predictions.append(output.item())

            # Update the input sequence by appending the latest prediction
            # Remove the first element and add the new prediction
            next_input = torch.cat((input_seq[:, 1:, :], output.unsqueeze(0).unsqueeze(0).repeat(1, 1, input_seq.size(2))), dim=1)
            input_seq = next_input

    # Inverse transform the predictions to the original scale
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)
    return future_predictions

# Parameters
num_future_steps = 10
input_size = len(x_train.columns)
hidden_size = 4
num_layers = 2
learning_rate = 0.0002
window_size = 5
forecast_horizon = 5

# Load your trained model
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, forecast_horizon)
# Assuming model is already trained, otherwise load the trained model

# Use the last window_size data points from the training set as the initial input
initial_input = x_train_normalized[-window_size:]

# Predict future steps
future_predictions = predict_future(model, initial_input, num_future_steps, scaler)

print("Future Predictions:", future_predictions)


TypeError: LSTMModel.__init__() takes 6 positional arguments but 7 were given

In [70]:
import numpy as np
# Define window size
window_size = 5

# Extract the last window from the training set
last_window = x_train[-window_size:]

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

initial_input = last_window
num_future_steps = 5
forecast_horizon = 5
scaler = MinMaxScaler()



def predict_future(model, initial_input, num_future_steps, scaler, device='cpu'):
    model.eval()
    future_predictions = []

    # Convert initial input to tensor and move to device
    input_seq = torch.tensor(initial_input, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        for _ in range(num_future_steps):
            output = model(input_seq)
            prediction = output[-1].item()  # Get the prediction for the last time step
            future_predictions.append(prediction)

            # Create the next input sequence by appending the latest prediction
            next_input = torch.cat((input_seq[:, 1:, :], output.unsqueeze(0)), dim=1)
            input_seq = next_input

    # Inverse transform the predictions to the original scale
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)
    return future_predictions


# Parameters
num_future_steps = 10
input_size = len(x_train.columns)
hidden_size = 4
num_layers = 2
learning_rate = 0.0002
window_size = 5
forecast_horizon = 5

# Load your trained model
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, forecast_horizon)
# Assuming model is already trained, otherwise load the trained model

# Use the last window_size data points from the training set as the initial input
initial_input = x_train_normalized[-window_size:]

# Predict future steps
future_predictions = predict_future(model, initial_input, num_future_steps, scaler)

print("Future Predictions:", future_predictions)


TypeError: LSTMModel.__init__() takes 6 positional arguments but 7 were given

In [ ]:
import numpy as np
# Define window size
window_size = 5

# Extract the last window from the training set
last_window = x_train[-window_size:]

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, forecast_horizon)

initial_input = last_window
num_future_steps = 5
forecast_horizon = 5
scaler = MinMaxScaler()



def predict_future(model, initial_input, num_future_steps, scaler, device='cpu'):
    model.eval()
    future_predictions = []

    # Convert initial input to tensor and move to device
    input_seq = torch.tensor(initial_input, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        for _ in range(num_future_steps):
            output = model(input_seq)
            prediction = output[-1].item()  # Get the prediction for the last time step
            future_predictions.append(prediction)

            # Create the next input sequence by appending the latest prediction
            next_input = torch.cat((input_seq[:, 1:, :], output.unsqueeze(0)), dim=1)
            input_seq = next_input

    # Inverse transform the predictions to the original scale
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)
    return future_predictions


# Parameters
num_future_steps = 10
input_size = len(x_train.columns)
hidden_size = 4
num_layers = 2
learning_rate = 0.0002
window_size = 5
forecast_horizon = 5

# Load your trained model
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, forecast_horizon)
# Assuming model is already trained, otherwise load the trained model

# Use the last window_size data points from the training set as the initial input
initial_input = x_train_normalized[-window_size:]

# Predict future steps
future_predictions = predict_future(model, initial_input, num_future_steps, scaler)

print("Future Predictions:", future_predictions)


TypeError: LSTMModel.__init__() takes 6 positional arguments but 7 were given

In [50]:
import numpy as np
# Define window size
window_size = 5

# Extract the last window from the training set
last_window = x_train[-window_size:]

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, forecast_horizon)
initial_input = last_window
num_future_steps = 5
forecast_horizon = 5
scaler = MinMaxScaler()



def predict_future(model, initial_input, num_future_steps, scaler, device='cpu'):
    model.eval()
    future_predictions = []

    # Convert initial input to tensor and move to device
    input_seq = torch.tensor(initial_input, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        for _ in range(num_future_steps):
            output = model(input_seq)
            future_predictions.append(output.item())

            # Create the next input sequence by appending the latest prediction
            next_input = torch.cat((input_seq[:, 1:, :], output.unsqueeze(0).unsqueeze(0)), dim=1)
            input_seq = next_input

    # Inverse transform the predictions to the original scale
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)
    return future_predictions

# Parameters
num_future_steps = 10
input_size = len(x_train.columns)
hidden_size = 4
num_layers = 2
learning_rate = 0.0002
window_size = 5
forecast_horizon = 5

# Load your trained model
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, forecast_horizon)
# Assuming model is already trained, otherwise load the trained model

# Use the last window_size data points from the training set as the initial input
initial_input = x_train_normalized[-window_size:]

# Predict future steps
future_predictions = predict_future(model, initial_input, num_future_steps, scaler)

print("Future Predictions:", future_predictions)


RuntimeError: a Tensor with 5 elements cannot be converted to Scalar